In [ ]:
#Load Module Gromacs
module load gromacs/openmpi/intel/2020.4

#Tutorial Starts
grep -v HOH 1aki.pdb > 1AKI_clean.pdb
gmx_mpi pdb2gmx -f 1AKI_clean.pdb -o 1AKI_processed.gro -water spce
#type 15,then type "enter"

#define box and solvate
gmx_mpi editconf -f 1AKI_processed.gro -o 1AKI_newbox.gro -c -d 1.0 -bt cubic

gmx_mpi solvate -cp 1AKI_newbox.gro -cs spc216.gro -o 1AKI_solv.gro -p topol.top

#add ions, energy minimization
wget http://www.mdtutorials.com/gmx/lysozyme/Files/ions.mdp
gmx_mpi grompp -f ions.mdp -c 1AKI_solv.gro -p topol.top -o ions.tpr
gmx_mpi genion -s ions.tpr -o 1AKI_solv_ions.gro -p topol.top -pname NA -nname CL -neutral
gmx_mpi grompp -f minim.mdp -c 1AKI_solv_ions.gro -p topol.top -o em.tpr
gmx_mpi mdrun -v -deffnm em
gmx_mpi energy -f em.edr -o potential.xvg

#equilibration
wget http://www.mdtutorials.com/gmx/lysozyme/Files/nvt.mdp
gmx_mpi grompp -f nvt.mdp -c em.gro -r em.gro -p topol.top -o nvt.tpr

gmx_mpi mdrun -deffnm nvt

gmx_mpi energy -f nvt.edr -o temperature.xvg
#type 16 0

#production MD
wget http://www.mdtutorials.com/gmx/lysozyme/Files/npt.mdp
gmx_mpi grompp -f npt.mdp -c nvt.gro -r nvt.gro -t nvt.cpt -p topol.top -o npt.tpr

gmx_mpi mdrun -deffnm npt
gmx_mpi energy -f npt.edr -o pressure.xvg
gmx_mpi energy -f npt.edr -o density.xvg
gmx_mpi grompp -f md.mdp -c npt.gro -t npt.cpt -p topol.top -o md_0_1.tpr
gmx_mpi mdrun -deffnm md_0_1
gmx_mpi mdrun -deffnm md_0_1 -nb gpu

